In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import ktrain
from ktrain import text


In [3]:
df1=pd.read_csv("IMDB dataset.csv")
df2=pd.read_csv("IMDB dataset.csv")

In [4]:
df1.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df1.groupby("sentiment").describe()

review                                                            \
           count unique                                                top   
sentiment                                                                    
negative   25000  24698  This show comes up with interesting locations ...   
positive   25000  24884  Loved today's show!!! It was a variety and not...   

                
          freq  
sentiment       
negative     3  
positive     5

In [6]:
# taking negative sentiment and deleting positive
df1 = df1[~df1.sentiment.str.contains("positive")]

In [7]:
df1.head()

,review,sentiment
3,Basically there's a family where a little boy ...,negative
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
10,Phil the Alien is one of those quirky films wh...,negative
11,I saw this movie when I was about 12 when it c...,negative


In [8]:
# randomly picking 5000 rows
df1 = df1.sample(n=5000, replace=True)

In [9]:
df1["sentiment"].value_counts()

negative    5000
Name: sentiment, dtype: int64

In [10]:
#taking positive senitment and deleting negative
df2 = df2[~df2.sentiment.str.contains("negative")]

In [11]:
df2.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive


In [12]:
# randomly picking 5000 rows
df2 = df2.sample(n=5000, replace=True)

In [13]:
df2["sentiment"].value_counts()

positive    5000
Name: sentiment, dtype: int64

In [14]:
#concating positive and negative sentiment
frames = [df1, df2]

df = pd.concat(frames)


In [15]:
df.head()

,review,sentiment
34957,I would have liked to put 0.5 but unfortunatel...,negative
12718,"No gore, no blood, no gratifying death scenes....",negative
49918,The film opens with a cult leader attempting t...,negative
38544,"This is truly, without exaggerating, one of th...",negative
37223,Back in 1994 the Power Rangers had become a hu...,negative


In [16]:
df["sentiment"].value_counts()

negative    5000
positive    5000
Name: sentiment, dtype: int64

In [17]:
class_names = ['positive', 'negative']

In [18]:
# replacing positive sentiment with 1 and negative with 0
encoding = {
    
    'positive': 1,
    'negative': 0
}

In [19]:
reviews = df['review'].values.tolist()
sentiments = df['sentiment'].tolist()

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(reviews, sentiments, test_size=.2)

In [21]:
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

In [22]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [23]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 350
done.


In [24]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [25]:
learner.fit_onecycle(2e-5,4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
1334/1334 [==============================] - 9238s 7s/step - loss: 0.3222 - accuracy: 0.8559 - val_loss: 0.2102 - val_accuracy: 0.9155
Epoch 2/4
1334/1334 [==============================] - 9235s 7s/step - loss: 0.1756 - accuracy: 0.9360 - val_loss: 0.2163 - val_accuracy: 0.9180
Epoch 3/4
1334/1334 [==============================] - 9241s 7s/step - loss: 0.0826 - accuracy: 0.9710 - val_loss: 0.1914 - val_accuracy: 0.9360
Epoch 4/4
1334/1334 [==============================] - 9395s 7s/step - loss: 0.0154 - accuracy: 0.9959 - val_loss: 0.2737 - val_accuracy: 0.9375


In [26]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

              precision    recall  f1-score   support

    positive       0.94      0.94      0.94      1010
    negative       0.94      0.94      0.94       990

    accuracy                           0.94      2000
   macro avg       0.94      0.94      0.94      2000
weighted avg       0.94      0.94      0.94      2000



array([[949,  61],
       [ 64, 926]], dtype=int64)

In [42]:
import time 

message = 'I dont feel so good as today is my exam'

start_time = time.time() 
prediction = predictor.predict(message)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))

predicted: negative (0.32)


In [40]:
predictor.save("models/bert_model")